In [215]:
import pandas as pd
pd.set_option("display.max_rows", 20)
igre = pd.read_csv("podatki_csv/igre.csv")

In [216]:
igre = igre.sort_values("Datum")
igre.reset_index()

,index,Datum,Verzija,Modra_Stran,Rdeca_Stran,Zmagovalec,Modri_Igralci,Rdeci_Igralci
0,1370,2014-01-14,3.15,Fnatic,Gambit Gaming,Fnatic,"['sOAZ', ' Cyanide', ' xPeke', ' Rekkles', ' Y...","['Darien', ' Diamond', ' Alex Ich', ' Genja', ..."
1,1369,2014-01-14,3.15,Copenhagen Wolves,Alliance,Copenhagen Wolves,"['YoungBuck', ' Amazing', ' cowTard', ' FORG1V...","['Wickd', ' Shook', ' Froggen', ' Tabzz', ' Ny..."
2,1368,2014-01-14,3.15,SUPA HOT CREW,Team ROCCAT,Team ROCCAT,"['Mimer', ' Impaler', ' Moopz', ' MrRalleZ', '...","['Xaxus', ' Jankos', ' Overpow', ' Celaver', '..."
3,1367,2014-01-14,3.15,Alliance,Fnatic,Fnatic,"['Wickd', ' Shook', ' Froggen', ' Tabzz', ' Ny...","['sOAZ', ' Cyanide', ' xPeke', ' Rekkles', ' Y..."
4,1366,2014-01-14,3.15,Gambit Gaming,Copenhagen Wolves,Gambit Gaming,"['Darien', ' Diamond', ' Alex Ich', ' Genja', ...","['YoungBuck', ' Amazing', ' cowTard', ' FORG1V..."
...,...,...,...,...,...,...,...,...
13225,7705,2024-08-03,14.14,DRX,BNK FearX,BNK FearX,"['Frog', ' Sponge', ' kyeahoo', ' Teddy', ' Pl...","['Clear', ' Raptor', ' Clozer', ' Hena', ' Duro']"
13226,7707,2024-08-03,14.14,T1,Gen.G,Gen.G,"['Zeus', ' Oner', ' Faker', ' Gumayusi', ' Ker...","['Kiin', ' Canyon', ' Chovy', ' Peyz', ' Lehen..."
13227,7708,2024-08-03,14.14,T1,Gen.G,Gen.G,"['Zeus', ' Oner', ' Faker', ' Gumayusi', ' Ker...","['Kiin', ' Canyon', ' Chovy', ' Peyz', ' Lehen..."
13228,12018,2024-08-03,14.14,Invictus Gaming,LGD Gaming,LGD Gaming,"['YSKM', ' glfs', ' neny', ' Ahn', ' Vampire']","['Burdol', ' Meteor', ' haichao', ' Shaoye', '..."


In [217]:
#nekatere ekipe so se preimenovale, zato želim vsa njihova imena upoštevati kot eno ekipo

preimenovane_ekipe = {'SK Telecom T1':'T1',
                      'DAMWON Gaming' : 'DWG KIA',
                      'KSV eSports' : 'Gen.G',
                      'Samsung Galaxy' : 'Gen.G',
                      'Afreeca Freecs':'Kwangdong Freecs'
}
igre['Modra_Stran'] = igre['Modra_Stran'].replace(preimenovane_ekipe)
igre['Rdeca_Stran'] = igre['Rdeca_Stran'].replace(preimenovane_ekipe)
igre['Zmagovalec'] = igre['Zmagovalec'].replace(preimenovane_ekipe)


In [218]:
stevilo_zmag = igre["Zmagovalec"].value_counts()

stevilo_zmag_tabela = stevilo_zmag.reset_index() 
stevilo_zmag_tabela.columns =['Ekipa', 'Zmage']

print(stevilo_zmag_tabela)

                  Ekipa  Zmage
0                    T1    599
1                 Gen.G    480
2         EDward Gaming    469
3                Cloud9    438
4                Fnatic    416
..                  ...    ...
164      KaBuM! Esports      1
165  CTBC Flying Oyster      1
166            Team Aze      1
167            INFINITY      1
168          DAN Gaming      1

[169 rows x 2 columns]


In [219]:
skupno = pd.concat([igre['Modra_Stran'], igre['Rdeca_Stran']]) #zdruzimo rdeci in modri stolpec
stevilo_iger = skupno.value_counts()
stevilo_iger_tabela = stevilo_iger.reset_index() 
stevilo_iger_tabela.columns =['Ekipa', 'Odigrane_Igre']
stevilo_iger_tabela

,Ekipa,Odigrane_Igre
0,T1,895
1,EDward Gaming,763
2,Gen.G,716
3,Cloud9,710
4,Fnatic,701
...,...,...
173,Fenerbahçe Esports,6
174,Hong Kong Attitude,6
175,CTBC Flying Oyster,6
176,Machi Esports,6


In [220]:
skupna = pd.merge(stevilo_zmag_tabela,stevilo_iger_tabela, on="Ekipa")
skupna['Odstotek_Zmag'] = (skupna['Zmage'] / skupna['Odigrane_Igre']) * 100
skupna['Odstotek_Zmag'] = skupna['Odstotek_Zmag'].round(2)
filtrirano = skupna[skupna['Odigrane_Igre'] >= 30]
filtrirano = filtrirano.sort_values("Odstotek_Zmag", ascending=False)
posamezno = filtrirano.loc[filtrirano['Ekipa'] == 'Invictus Gaming']
posamezno

,Ekipa,Zmage,Odigrane_Igre,Odstotek_Zmag
10,Invictus Gaming,339,650,52.15


In [221]:
elo_ratings = {}
zacetni = 1000
K_faktor = 32

def prva_igra(ekipa):
    if ekipa not in elo_ratings:
        elo_ratings[ekipa] = zacetni

def predviden_rezultat(rating1, rating2):
    return 1 / (1 + 10 ** ((rating2 - rating1) / 400))

def posodobi_elo(zmagovalec, porazenec):
    zmaga = predviden_rezultat(elo_ratings[zmagovalec], elo_ratings[porazenec])
    poraz = 1 - zmaga

    elo_ratings[zmagovalec] += K_faktor * (1 - zmaga)
    elo_ratings[porazenec] += K_faktor * (0 - poraz)

for index, row in igre.iterrows():
    team1 = row['Modra_Stran']
    team2 = row['Rdeca_Stran']
    winner = row['Zmagovalec']
    loser = team2 if winner == team1 else team1
    prva_igra(team1)
    prva_igra(team2)
    posodobi_elo(winner, loser)

elo_ratings_df = pd.DataFrame(elo_ratings.items(), columns=['Ekipa', 'Elo'])

elo_ratings_df = elo_ratings_df.sort_values('Elo')
elo_ratings_df


,Ekipa,Elo
11,Team Coast,775.054767
74,Jin Air Green Wings,798.948257
137,EStar (Chinese Team),807.435183
140,SeolHaeOne Prince,809.315956
69,Immortals,812.646914
...,...,...
126,DWG KIA,1278.031486
117,Hanwha Life Esports,1319.268123
113,Bilibili Gaming,1326.219272
128,Top Esports,1334.685803
